In [132]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings 
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import joblib

from collections import Counter
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import nltk
import spacy
import gensim
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import sent_tokenize

import statistics


import re
from os import path
from PIL import Image
import plotly
from sklearn.feature_extraction.text import CountVectorizer

import nltk
from nltk.collocations import *
nltk.download('averaged_perceptron_tagger')

from textblob import TextBlob

import seaborn as sns

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/sameer/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [133]:
def normalize_list(data):
    x_min = min(data)
    x_max = max(data)
    if x_max-x_min == 0:
        return None
    return [(x-x_min)/(x_max-x_min) for x in data]

In [134]:
df = pd.read_csv('./meta_sentiments.csv', encoding='utf-8')

In [135]:
df.head()

,Unnamed: 0,Company,Star_Rating,Current_Employee,Work_Duration,Review_Title,Date_Posted,Job_Title,Job_Location,Recommend,CEO_Approval,Business_Outlook,Pros,Cons,Advice_to_Management,Pros_And_Cons,Review_Title_sentiment_score_vader,Pros_And_Cons_sentiment_score_vader
0,0,Meta,5.0,1,less than 1 year,Best employer so far,2021-11-09,Software Engineer,"Seattle, WA",2,2,2,"I’m still a bit new, but I have very good impr...",Some risk with new company direction. The meta...,NaN,"I’m still a bit new, but I have very good impr...",0.64,1.00
1,1,Meta,5.0,1,NaN,Great place,2022-02-26,Research Manager,"Miami, FL",2,2,2,Challenging work and great salary,Lots of new people- culture is getting diluted,NaN,Challenging work and great salaryLots of new p...,0.62,0.69
2,2,Meta,5.0,1,more than 1 year,Visionary and Innovative,2022-02-25,Software Engineer,"New York, NY",2,2,2,The management is efficient and the vision is ...,"It's a big company, so you'll likely be workin...",NaN,The management is efficient and the vision is ...,0.74,0.90
3,3,Meta,5.0,1,more than 3 years,Great company to work for,2022-02-24,Program Manager,"New York, NY",2,2,2,"Amazing company, so many smart, kind people an...","Bad media headlines, but that doesn't distract...",NaN,"Amazing company, so many smart, kind people an...",0.62,0.79
4,4,Meta,5.0,1,more than 1 year,good,2022-02-24,Data Scientist,"Boston, MA",2,2,2,"good to work , and gives you good experience.",the work load is quite heavy,can improve work culture,"good to work , and gives you good experience.t...",0.44,0.70


In [136]:
df.columns

Index(['Unnamed: 0', 'Company', 'Star_Rating', 'Current_Employee', 'Work_Duration', 'Review_Title', 'Date_Posted', 'Job_Title', 'Job_Location', 'Recommend', 'CEO_Approval', 'Business_Outlook', 'Pros', 'Cons', 'Advice_to_Management', 'Pros_And_Cons', 'Review_Title_sentiment_score_vader', 'Pros_And_Cons_sentiment_score_vader'], dtype='object')

In [137]:
df.drop(['Unnamed: 0', 'Company'], axis=1, inplace=True)

In [138]:
df.head()

,Star_Rating,Current_Employee,Work_Duration,Review_Title,Date_Posted,Job_Title,Job_Location,Recommend,CEO_Approval,Business_Outlook,Pros,Cons,Advice_to_Management,Pros_And_Cons,Review_Title_sentiment_score_vader,Pros_And_Cons_sentiment_score_vader
0,5.0,1,less than 1 year,Best employer so far,2021-11-09,Software Engineer,"Seattle, WA",2,2,2,"I’m still a bit new, but I have very good impr...",Some risk with new company direction. The meta...,NaN,"I’m still a bit new, but I have very good impr...",0.64,1.00
1,5.0,1,NaN,Great place,2022-02-26,Research Manager,"Miami, FL",2,2,2,Challenging work and great salary,Lots of new people- culture is getting diluted,NaN,Challenging work and great salaryLots of new p...,0.62,0.69
2,5.0,1,more than 1 year,Visionary and Innovative,2022-02-25,Software Engineer,"New York, NY",2,2,2,The management is efficient and the vision is ...,"It's a big company, so you'll likely be workin...",NaN,The management is efficient and the vision is ...,0.74,0.90
3,5.0,1,more than 3 years,Great company to work for,2022-02-24,Program Manager,"New York, NY",2,2,2,"Amazing company, so many smart, kind people an...","Bad media headlines, but that doesn't distract...",NaN,"Amazing company, so many smart, kind people an...",0.62,0.79
4,5.0,1,more than 1 year,good,2022-02-24,Data Scientist,"Boston, MA",2,2,2,"good to work , and gives you good experience.",the work load is quite heavy,can improve work culture,"good to work , and gives you good experience.t...",0.44,0.70


In [139]:
df['Date_Posted'] = pd.to_datetime(df['Date_Posted'], dayfirst=True)

In [140]:
quarters = pd.Series(pd.date_range('2012-01-01', '2022-01-01', freq='QS'))

In [141]:
len(quarters)

41

In [142]:
for i, row in df.iterrows():
	if isinstance(row['Work_Duration'], str):
		if 'less' in row['Work_Duration']:
			years = [int(i) for i in row['Work_Duration'].split() if i.isdigit()]
			df.at[i, 'Work_Duration'] = years[0] - 1
			# row['Work_Duration'] = years[0] - 1
		elif 'more' in row['Work_Duration']:
			years = [int(i) for i in row['Work_Duration'].split() if i.isdigit()]
			df.at[i, 'Work_Duration'] = years[0]
			# row['Work_Duration'] = years[0]

In [143]:
df.head()

,Star_Rating,Current_Employee,Work_Duration,Review_Title,Date_Posted,Job_Title,Job_Location,Recommend,CEO_Approval,Business_Outlook,Pros,Cons,Advice_to_Management,Pros_And_Cons,Review_Title_sentiment_score_vader,Pros_And_Cons_sentiment_score_vader
0,5.0,1,0,Best employer so far,2021-11-09,Software Engineer,"Seattle, WA",2,2,2,"I’m still a bit new, but I have very good impr...",Some risk with new company direction. The meta...,NaN,"I’m still a bit new, but I have very good impr...",0.64,1.00
1,5.0,1,NaN,Great place,2022-02-26,Research Manager,"Miami, FL",2,2,2,Challenging work and great salary,Lots of new people- culture is getting diluted,NaN,Challenging work and great salaryLots of new p...,0.62,0.69
2,5.0,1,1,Visionary and Innovative,2022-02-25,Software Engineer,"New York, NY",2,2,2,The management is efficient and the vision is ...,"It's a big company, so you'll likely be workin...",NaN,The management is efficient and the vision is ...,0.74,0.90
3,5.0,1,3,Great company to work for,2022-02-24,Program Manager,"New York, NY",2,2,2,"Amazing company, so many smart, kind people an...","Bad media headlines, but that doesn't distract...",NaN,"Amazing company, so many smart, kind people an...",0.62,0.79
4,5.0,1,1,good,2022-02-24,Data Scientist,"Boston, MA",2,2,2,"good to work , and gives you good experience.",the work load is quite heavy,can improve work culture,"good to work , and gives you good experience.t...",0.44,0.70


In [144]:
df.drop('Job_Title', axis=1, inplace=True)

In [145]:
df.drop('Advice_to_Management', axis=1, inplace=True)

In [146]:
df.drop('Job_Location', axis=1, inplace=True)

In [147]:
df['Work_Duration'].fillna(df['Work_Duration'].mean(),inplace=True)

# other value aggregates

In [148]:
star_rating_aggregates = dict()
title_senti_aggregates = dict()
pnc_senti_aggregates = dict()
work_duration_aggregates = dict()

current_employee_aggregates = dict()
recommend_aggregates = dict()
ceo_approval_aggregates = dict()
business_outlook_aggregates = dict()



# percentages
recommend_percentages = dict()
ceo_percentages = dict()
business_percentages = dict()
counts = dict()
count = 0

In [149]:

for q in quarters:
	star_rating_aggregates[f'Q{count+1}'] = 0
	title_senti_aggregates[f'Q{count+1}'] = 0
	pnc_senti_aggregates[f'Q{count+1}'] = 0
	work_duration_aggregates[f'Q{count+1}'] = 0


	current_employee_aggregates[f'Q{count+1}'] = []
	recommend_aggregates[f'Q{count+1}'] = []
	ceo_approval_aggregates[f'Q{count+1}'] = []
	business_outlook_aggregates[f'Q{count+1}'] = []

	recommend_percentages[f'Q{count+1}'] = dict()
	ceo_percentages[f'Q{count+1}'] = dict()
	business_percentages[f'Q{count+1}'] = dict()
	for i in range(-1, 3):
		recommend_percentages[f'Q{count+1}'][i] = 0 
		ceo_percentages[f'Q{count+1}'][i] = 0
		business_percentages[f'Q{count+1}'][i] = 0


	counts[f'Q{count+1}'] = 0
	count += 1

for idx, row in df.iterrows():
	for i in range(len(quarters)):
		if i < len(quarters) - 1:
			if row['Date_Posted'] >= quarters[i] and row['Date_Posted'] < quarters[i+1]:
				star_rating_aggregates[f'Q{i+1}'] += row['Star_Rating']
				title_senti_aggregates[f'Q{i+1}'] += row['Review_Title_sentiment_score_vader']
				pnc_senti_aggregates[f'Q{i+1}'] += row['Pros_And_Cons_sentiment_score_vader']
				work_duration_aggregates[f'Q{i+1}'] += row['Work_Duration']


				current_employee_aggregates[f'Q{i+1}'].append(row['Current_Employee'])
				recommend_aggregates[f'Q{i+1}'].append(row['Recommend'])
				ceo_approval_aggregates[f'Q{i+1}'].append(row['CEO_Approval'])
				business_outlook_aggregates[f'Q{i+1}'].append(row['Business_Outlook'])

				recommend_percentages[f'Q{i+1}'][row['Recommend']] += 1
				ceo_percentages[f'Q{i+1}'][row['CEO_Approval']] += 1
				business_percentages[f'Q{i+1}'][row['Business_Outlook']] += 1
				
				counts[f'Q{i+1}'] += 1

In [150]:

avg_star_ratings = []
avg_title_senti = []
avg_pnc_senti = []
avg_work_duration = []

avg_current_employee = []
avg_recommend = []
avg_ceo_approval = []
avg_business_outlook = []

# percentages 

recommend_minus_percentages = []
ceo_minus_percentages = []
business_minus_percentages = []

recommend_unknown_percentages = []
ceo_unknown_percentages = []
business_unknown_percentages = []

recommend_same_percentages = []
ceo_same_percentages = []
business_same_percentages = []

recommend_plus_percentages = []
ceo_plus_percentages = []
business_plus_percentages = []


quarts = []


In [151]:
for quarter, value in star_rating_aggregates.items():

	if counts[quarter] != 0:

		avg_star_ratings.append(star_rating_aggregates[quarter] / counts[quarter])
		avg_title_senti.append(title_senti_aggregates[quarter] / counts[quarter])
		avg_pnc_senti.append(pnc_senti_aggregates[quarter] / counts[quarter])
		avg_work_duration.append(work_duration_aggregates[quarter] / counts[quarter])


		avg_current_employee.append(statistics.mode(current_employee_aggregates[quarter]))
		avg_recommend.append(statistics.mode(recommend_aggregates[quarter]))
		avg_ceo_approval.append(statistics.mode(ceo_approval_aggregates[quarter]))
		avg_business_outlook.append(statistics.mode(business_outlook_aggregates[quarter]))

		recommend_minus_percentages.append(recommend_percentages[quarter][-1] / counts[quarter])
		ceo_minus_percentages.append(ceo_percentages[quarter][-1] / counts[quarter])
		business_minus_percentages.append(business_percentages[quarter][-1] / counts[quarter])

		recommend_unknown_percentages.append(recommend_percentages[quarter][0] / counts[quarter])
		ceo_unknown_percentages.append(ceo_percentages[quarter][0] / counts[quarter])
		business_unknown_percentages.append(business_percentages[quarter][0] / counts[quarter])

		recommend_same_percentages.append(recommend_percentages[quarter][1] / counts[quarter])
		ceo_same_percentages.append(ceo_percentages[quarter][1] / counts[quarter])
		business_same_percentages.append(business_percentages[quarter][1] / counts[quarter])

		recommend_plus_percentages.append(recommend_percentages[quarter][2] / counts[quarter])
		ceo_plus_percentages.append(ceo_percentages[quarter][2] / counts[quarter])
		business_plus_percentages.append(business_percentages[quarter][2] / counts[quarter])
		
		quarts.append(quarter)

<hr>

In [152]:
def clean_text(text):
	text=re.sub(r"\?", ".", text)
	text=re.sub(r"\!", ".", text)
	text=re.sub(r'([.])\1+', r'\1', text)
	rexp=r"\.(?=[A-Za-z]{1,15})"
	text=re.sub(rexp, ". ", text)
	return text

#this function extracts lemmas of nouns, verbs and adjectives
def lemma_words_pos_filtered(text):
    word_list=[]
    lemmatizer = WordNetLemmatizer()
    tokens = nltk.word_tokenize(text)
    pos_tokens=nltk.pos_tag(tokens)
    for pos in pos_tokens:
        if (pos[1].startswith('N')):
            word_list=word_list+[lemmatizer.lemmatize(pos[0].lower(), wordnet.NOUN)]
        if (pos[1].startswith('V')):
            word_list=word_list+[lemmatizer.lemmatize(pos[0].lower(), wordnet.VERB)]
        if (pos[1].startswith('J')):
            word_list=word_list+[lemmatizer.lemmatize(pos[0].lower(), wordnet.ADJ)]
    word_list=[word for word in word_list if word not in  
                                    stopwords.words('english') ]
    return  " ".join(word_list)

#to keep only unique tuples
def unique_tuples(list_of_tuples):
     list_ = [tuple(sorted(t)) for t in list_of_tuples]
     list_ = list(set(list_))
     return list_

def findbigramsintext(text):
	mylist=nltk.word_tokenize(text)
	list1=[x for x in mylist]
	feature_list = []
	for i in range(length_features2):
	# In each iteration, add an empty list to the main list      
			feature_list.append([])
	i=0
	# bigram_features is the list of unique set of bigrams found after collocation
	for l in bigram_features:
			list2=[x for x in l]
			result =  all(elem in list1  for elem in list2)
			if result: 
				feature_list[i].append(' '.join(list2))
			i=i+1
	return feature_list


def keepnonempty(list1):
       mylist= [x for x in list1 if x != []]
       return mylist
def flatten_list(row_list):
        l = [item for inner_list in row_list for item in inner_list]
        return l


In [153]:
import nltk
nltk.download('punkt')
nltk.download('wordnet') 
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/sameer/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/sameer/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sameer/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# text cleaning

In [154]:

df['Cleaned_Pros_and_Cons_Text'] = df['Pros_And_Cons'].apply(sent_tokenize)

In [155]:
len(df)

4854

In [156]:
df = df.explode('Cleaned_Pros_and_Cons_Text', ignore_index=False)

In [157]:
len(df)

11930

In [158]:
df['lemma_words_pos_filtered'] = df['Cleaned_Pros_and_Cons_Text'].apply(lemma_words_pos_filtered)

In [159]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_words( " ".join(df['lemma_words_pos_filtered']).split(),window_size=5)
#keeping bigrams that occur at least 5 times
finder.apply_freq_filter(5)
bigram_list_tuples=finder.nbest(bigram_measures.likelihood_ratio, 20)

In [160]:
bigram_list_tuples

[('life', 'balance'),
 ('free', 'food'),
 ('none', 'none'),
 ('smart', 'people'),
 ('work', 'life'),
 ('work-life', 'balance'),
 ('performance', 'review'),
 ('work/life', 'balance'),
 ('*', '*'),
 ('menlo', 'park'),
 ('fast', 'pace'),
 ('great', 'benefit'),
 ('career', 'growth'),
 ('work', 'balance'),
 ('take', 'care'),
 ('open', 'culture'),
 ('authentic', 'self'),
 ('place', 'work'),
 ('internal', 'tool'),
 ('great', 'perk')]

In [161]:
bigram_features=unique_tuples(bigram_list_tuples)
length_features2=len(bigram_features)

In [162]:
df['bigram_list'] = df['lemma_words_pos_filtered'].apply(findbigramsintext)

In [163]:
df['bigrams']=df['bigram_list'].apply(keepnonempty)
df['bigrams']=df['bigrams'].apply(flatten_list)

In [164]:
#Sentiment scores are calculated using libraries TextBlob and Vader
df['polarity'] = df['Cleaned_Pros_and_Cons_Text'].apply(lambda x: TextBlob(x).sentiment[0])

In [165]:
df.head()

,Star_Rating,Current_Employee,Work_Duration,Review_Title,Date_Posted,Recommend,CEO_Approval,Business_Outlook,Pros,Cons,Pros_And_Cons,Review_Title_sentiment_score_vader,Pros_And_Cons_sentiment_score_vader,Cleaned_Pros_and_Cons_Text,lemma_words_pos_filtered,bigram_list,bigrams,polarity
0,5.0,1,0.0,Best employer so far,2021-11-09,2,2,2,"I’m still a bit new, but I have very good impr...",Some risk with new company direction. The meta...,"I’m still a bit new, but I have very good impr...",0.64,1.0,"I’m still a bit new, but I have very good impr...",’ bit new good impression,"[[], [], [], [], [], [], [], [], [], [], [], [...",[],0.382121
0,5.0,1,0.0,Best employer so far,2021-11-09,2,2,2,"I’m still a bit new, but I have very good impr...",Some risk with new company direction. The meta...,"I’m still a bit new, but I have very good impr...",0.64,1.0,I have 10 years exp at 5+ other companies (alt...,year company tech company best good margin,"[[], [], [], [], [], [], [], [], [], [], [], [...",[],0.409375
0,5.0,1,0.0,Best employer so far,2021-11-09,2,2,2,"I’m still a bit new, but I have very good impr...",Some risk with new company direction. The meta...,"I’m still a bit new, but I have very good impr...",0.64,1.0,"First off, everyone is nice here!",first everyone nice,"[[], [], [], [], [], [], [], [], [], [], [], [...",[],0.500000
0,5.0,1,0.0,Best employer so far,2021-11-09,2,2,2,"I’m still a bit new, but I have very good impr...",Some risk with new company direction. The meta...,"I’m still a bit new, but I have very good impr...",0.64,1.0,"Yes, folks are ambitious, but there is a shock...",folk ambitious small percentage jerk here… sur...,"[[], [], [], [], [], [], [], [], [], [], [], [...",[],0.015625
0,5.0,1,0.0,Best employer so far,2021-11-09,2,2,2,"I’m still a bit new, but I have very good impr...",Some risk with new company direction. The meta...,"I’m still a bit new, but I have very good impr...",0.64,1.0,Lots of small benefits and well-run systems gi...,lot small benefit well-run system give overwhe...,"[[], [], [* *], [], [], [], [], [], [], [], []...",[* *],0.100000


In [166]:
copy_df = df.copy()

In [167]:
copy_df = copy_df.explode('bigrams', ignore_index=False)

In [168]:
res = copy_df[['bigrams', 'polarity']].groupby('bigrams', as_index = False)['polarity'].mean()

In [169]:
print(res)

               bigrams  polarity
0                  * *  0.222564
1       authentic self  0.405843
2         balance life  0.259605
3         balance work  0.250308
4    balance work-life  0.228875
5    balance work/life  0.163387
6        benefit great  0.477557
7            care take  0.242306
8        career growth  0.240431
9         culture open  0.201671
10           fast pace  0.219445
11           food free  0.338149
12          great perk  0.504882
13       internal tool  0.193387
14           life work  0.264789
15          menlo park  0.179162
16           none none  0.300243
17        people smart  0.280375
18  performance review  0.164885
19          place work  0.449332


In [170]:
all_bigrams = res['bigrams'].tolist()
print(all_bigrams)

['* *', 'authentic self', 'balance life', 'balance work', 'balance work-life', 'balance work/life', 'benefit great', 'care take', 'career growth', 'culture open', 'fast pace', 'food free', 'great perk', 'internal tool', 'life work', 'menlo park', 'none none', 'people smart', 'performance review', 'place work']


In [171]:
result = copy_df[['bigrams', 'polarity']].groupby('bigrams')['polarity'].mean()

In [172]:
topic_polarities = dict()
for b in all_bigrams:
	topic_polarities[b] = []

In [173]:
polarity_results = []
for i in range(len(quarters)):
	if i < len(quarters) - 1:
		qtr_start = quarters[i]
		qtr_end = quarters[i+1]
		cur_df = df[(df['Date_Posted'] >= qtr_start) & (df['Date_Posted'] < qtr_end)]
		if(len(cur_df) == 0):
			continue
		cur_df = cur_df.explode('bigrams', ignore_index=False)
		results = cur_df[['bigrams', 'polarity']].groupby('bigrams', as_index = False)['polarity'].mean()

		for bigram in all_bigrams:
			if bigram in results['bigrams'].tolist():
				polarity = results[results['bigrams'] == bigram]['polarity'].tolist()[0]
				topic_polarities[bigram].append(polarity)
				# polarity_results.append(results[results['bigrams'] == bigram]['polarity'].tolist()[0])
			else:
				topic_polarities[bigram].append(None)

		# for i, row in results.iterrows():
		# 	if row['bigrams'] in topic_polarities:
		# 		topic_polarities[row['bigrams']].append(row['polarity'])
		# 	else:
		# 		topic_polarities[row['bigrams']] = [row['polarity']]
		polarity_results.append(results)


In [174]:
len(topic_polarities)

20

In [175]:
len(polarity_results)

40

In [176]:
for key, value in topic_polarities.items():
	print(key, len(value))

* * 40
authentic self 40
balance life 40
balance work 40
balance work-life 40
balance work/life 40
benefit great 40
care take 40
career growth 40
culture open 40
fast pace 40
food free 40
great perk 40
internal tool 40
life work 40
menlo park 40
none none 40
people smart 40
performance review 40
place work 40


In [177]:
print(counts)

{'Q1': 6, 'Q2': 7, 'Q3': 23, 'Q4': 24, 'Q5': 7, 'Q6': 4, 'Q7': 11, 'Q8': 19, 'Q9': 18, 'Q10': 14, 'Q11': 19, 'Q12': 12, 'Q13': 26, 'Q14': 69, 'Q15': 68, 'Q16': 85, 'Q17': 83, 'Q18': 58, 'Q19': 79, 'Q20': 87, 'Q21': 138, 'Q22': 67, 'Q23': 100, 'Q24': 95, 'Q25': 92, 'Q26': 106, 'Q27': 111, 'Q28': 94, 'Q29': 95, 'Q30': 108, 'Q31': 113, 'Q32': 92, 'Q33': 122, 'Q34': 196, 'Q35': 255, 'Q36': 295, 'Q37': 639, 'Q38': 467, 'Q39': 347, 'Q40': 319, 'Q41': 0}


In [178]:
final = pd.DataFrame(
{   'quarters': quarts,
	'ratings': normalize_list(avg_star_ratings),
	'title_senti': normalize_list(avg_title_senti),
	'pnc_senti': normalize_list(avg_pnc_senti),
	'work_duration': normalize_list(avg_work_duration),
	'current_employee': avg_current_employee,
	'recommend': avg_recommend,
	'ceo_approval': avg_ceo_approval,
	'business_outlook': avg_business_outlook,
	'recommend_minus': normalize_list(recommend_minus_percentages),
	'ceo_minus': normalize_list(ceo_minus_percentages),
	'business_minus': normalize_list(business_minus_percentages),
	'recommend_unknown': normalize_list(recommend_unknown_percentages),
	'ceo_unknown': normalize_list(ceo_unknown_percentages),
	'business_unknown': normalize_list(business_unknown_percentages),
	'recommend_same': normalize_list(recommend_same_percentages),
	'ceo_same': normalize_list(ceo_same_percentages),
	'business_same': normalize_list(business_same_percentages),
	'recommend_plus': normalize_list(recommend_plus_percentages),
	'ceo_plus': normalize_list(ceo_plus_percentages),
	'business_plus': normalize_list(business_plus_percentages),
})

In [179]:
final

,quarters,ratings,title_senti,pnc_senti,work_duration,current_employee,recommend,ceo_approval,business_outlook,recommend_minus,ceo_minus,business_minus,recommend_unknown,ceo_unknown,business_unknown,recommend_same,ceo_same,business_same,recommend_plus,ceo_plus,business_plus
0,Q1,0.951049,0.800650,0.840840,0.768193,1,2,2,0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,None,0.000000,0.000000,1.000000,1.000000,0.000000
1,Q2,0.839161,0.654483,0.706417,0.931246,1,2,2,2,0.500000,0.000000,0.000000,0.000000,0.000000,0.428571,None,0.000000,0.000000,0.711573,1.000000,0.623377
2,Q3,0.831864,0.406544,0.606447,0.229029,1,2,2,2,0.152174,0.304348,0.227053,0.000000,0.000000,0.000000,None,0.000000,0.284858,0.912218,0.925029,0.948617
3,Q4,1.000000,0.777833,0.158513,0.402000,1,2,2,2,0.291667,0.000000,0.000000,0.119376,0.000000,0.041667,None,0.166667,0.136494,0.747627,0.856306,1.000000
4,Q5,0.000000,0.456848,0.122679,0.537518,1,2,2,2,1.000000,1.000000,0.746032,0.000000,0.000000,0.000000,None,0.000000,0.467980,0.423146,0.753668,0.779221
5,Q6,0.167832,0.461995,0.656287,0.150505,1,2,2,2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,None,1.000000,0.818966,1.000000,0.137838,0.818182
6,Q7,0.381437,0.904146,0.557748,0.945501,1,2,2,2,0.318182,0.636364,0.000000,0.000000,0.000000,0.090909,None,0.363636,0.893417,0.816456,0.529730,0.694215
7,Q8,0.538830,0.765002,0.531929,0.245561,1,2,2,2,0.552632,0.368421,0.274854,0.000000,0.000000,0.052632,None,0.000000,0.172414,0.681213,0.909246,0.918660
8,Q9,0.820513,0.339503,0.073168,0.000000,0,2,2,2,0.388889,0.000000,0.000000,0.000000,0.137272,0.166667,None,0.111111,0.545977,0.775668,0.808408,0.727273
9,Q10,0.083916,0.473318,0.220711,0.602020,1,2,2,2,0.500000,0.500000,0.373016,0.204645,0.352986,0.142857,None,0.285714,0.935961,0.567360,0.384170,0.545455


In [180]:
all_bigrams

['* *',
 'authentic self',
 'balance life',
 'balance work',
 'balance work-life',
 'balance work/life',
 'benefit great',
 'care take',
 'career growth',
 'culture open',
 'fast pace',
 'food free',
 'great perk',
 'internal tool',
 'life work',
 'menlo park',
 'none none',
 'people smart',
 'performance review',
 'place work']

In [181]:
for key,value in topic_polarities.items():
	col_name = key.replace(' ', '_')
	final[col_name] = value

In [182]:
final

,quarters,ratings,title_senti,pnc_senti,work_duration,current_employee,recommend,ceo_approval,business_outlook,recommend_minus,ceo_minus,business_minus,recommend_unknown,ceo_unknown,business_unknown,recommend_same,ceo_same,business_same,recommend_plus,ceo_plus,business_plus,*_*,authentic_self,balance_life,balance_work,balance_work-life,balance_work/life,benefit_great,care_take,career_growth,culture_open,fast_pace,food_free,great_perk,internal_tool,life_work,menlo_park,none_none,people_smart,performance_review,place_work
0,Q1,0.951049,0.800650,0.840840,0.768193,1,2,2,0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,None,0.000000,0.000000,1.000000,1.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.452827,0.427083,NaN,NaN,0.478571,NaN,0.639286,NaN,NaN,NaN,NaN,0.239286,NaN,NaN
1,Q2,0.839161,0.654483,0.706417,0.931246,1,2,2,2,0.500000,0.000000,0.000000,0.000000,0.000000,0.428571,None,0.000000,0.000000,0.711573,1.000000,0.623377,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.587500,NaN,0.266667,NaN,NaN,NaN,NaN,0.062500
2,Q3,0.831864,0.406544,0.606447,0.229029,1,2,2,2,0.152174,0.304348,0.227053,0.000000,0.000000,0.000000,None,0.000000,0.284858,0.912218,0.925029,0.948617,0.277778,NaN,0.086111,0.014931,NaN,NaN,0.543750,NaN,0.366667,NaN,0.200000,0.308452,0.554857,NaN,-0.027778,NaN,NaN,0.341929,0.352083,0.634766
3,Q4,1.000000,0.777833,0.158513,0.402000,1,2,2,2,0.291667,0.000000,0.000000,0.119376,0.000000,0.041667,None,0.166667,0.136494,0.747627,0.856306,1.000000,NaN,NaN,0.520000,0.520000,NaN,-0.062500,0.361756,NaN,NaN,0.200000,0.050000,0.207292,0.361756,NaN,0.520000,NaN,NaN,0.283534,NaN,0.550000
4,Q5,0.000000,0.456848,0.122679,0.537518,1,2,2,2,1.000000,1.000000,0.746032,0.000000,0.000000,0.000000,None,0.000000,0.467980,0.423146,0.753668,0.779221,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.650000,NaN,NaN,NaN,NaN,0.608333
5,Q6,0.167832,0.461995,0.656287,0.150505,1,2,2,2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,None,1.000000,0.818966,1.000000,0.137838,0.818182,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Q7,0.381437,0.904146,0.557748,0.945501,1,2,2,2,0.318182,0.636364,0.000000,0.000000,0.000000,0.090909,None,0.363636,0.893417,0.816456,0.529730,0.694215,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,0.216288,0.523125,NaN,0.216288,NaN,NaN,NaN,0.200000,NaN,NaN
7,Q8,0.538830,0.765002,0.531929,0.245561,1,2,2,2,0.552632,0.368421,0.274854,0.000000,0.000000,0.052632,None,0.000000,0.172414,0.681213,0.909246,0.918660,NaN,NaN,NaN,NaN,NaN,0.000000,0.537500,NaN,NaN,NaN,NaN,0.533333,0.618019,NaN,-0.250000,0.172078,NaN,NaN,NaN,0.750000
8,Q9,0.820513,0.339503,0.073168,0.000000,0,2,2,2,0.388889,0.000000,0.000000,0.000000,0.137272,0.166667,None,0.111111,0.545977,0.775668,0.808408,0.727273,NaN,NaN,0.300000,0.300000,NaN,NaN,NaN,0.050000,NaN,0.193056,0.566667,0.342857,0.270941,0.450000,0.550000,NaN,NaN,0.212572,NaN,0.496528
9,Q10,0.083916,0.473318,0.220711,0.602020,1,2,2,2,0.500000,0.500000,0.373016,0.204645,0.352986,0.142857,None,0.285714,0.935961,0.567360,0.384170,0.545455,NaN,NaN,-0.520000,-0.520000,NaN,0.102083,NaN,0.625000,NaN,NaN,0.226000,NaN,0.580000,NaN,-0.520000,NaN,NaN,NaN,NaN,0.560000


In [183]:
for key, value in topic_polarities.items():
	col = key.replace(' ', '_')
	final[col].fillna(final[col].mean(),inplace=True)

In [184]:
final

,quarters,ratings,title_senti,pnc_senti,work_duration,current_employee,recommend,ceo_approval,business_outlook,recommend_minus,ceo_minus,business_minus,recommend_unknown,ceo_unknown,business_unknown,recommend_same,ceo_same,business_same,recommend_plus,ceo_plus,business_plus,*_*,authentic_self,balance_life,balance_work,balance_work-life,balance_work/life,benefit_great,care_take,career_growth,culture_open,fast_pace,food_free,great_perk,internal_tool,life_work,menlo_park,none_none,people_smart,performance_review,place_work
0,Q1,0.951049,0.800650,0.840840,0.768193,1,2,2,0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,None,0.000000,0.000000,1.000000,1.000000,0.000000,0.201553,0.431776,0.212980,0.197778,0.220602,0.130941,0.452827,0.427083,0.276800,0.212379,0.478571,0.357914,0.639286,0.180615,0.226630,0.139445,0.277113,0.239286,0.174050,0.429060
1,Q2,0.839161,0.654483,0.706417,0.931246,1,2,2,2,0.500000,0.000000,0.000000,0.000000,0.000000,0.428571,None,0.000000,0.000000,0.711573,1.000000,0.623377,0.201553,0.431776,0.212980,0.197778,0.220602,0.130941,0.489542,0.252479,0.276800,0.212379,0.230337,0.357914,0.587500,0.180615,0.266667,0.139445,0.277113,0.283738,0.174050,0.062500
2,Q3,0.831864,0.406544,0.606447,0.229029,1,2,2,2,0.152174,0.304348,0.227053,0.000000,0.000000,0.000000,None,0.000000,0.284858,0.912218,0.925029,0.948617,0.277778,0.431776,0.086111,0.014931,0.220602,0.130941,0.543750,0.252479,0.366667,0.212379,0.200000,0.308452,0.554857,0.180615,-0.027778,0.139445,0.277113,0.341929,0.352083,0.634766
3,Q4,1.000000,0.777833,0.158513,0.402000,1,2,2,2,0.291667,0.000000,0.000000,0.119376,0.000000,0.041667,None,0.166667,0.136494,0.747627,0.856306,1.000000,0.201553,0.431776,0.520000,0.520000,0.220602,-0.062500,0.361756,0.252479,0.276800,0.200000,0.050000,0.207292,0.361756,0.180615,0.520000,0.139445,0.277113,0.283534,0.174050,0.550000
4,Q5,0.000000,0.456848,0.122679,0.537518,1,2,2,2,1.000000,1.000000,0.746032,0.000000,0.000000,0.000000,None,0.000000,0.467980,0.423146,0.753668,0.779221,0.201553,0.431776,0.212980,0.197778,0.220602,0.130941,0.489542,0.252479,0.276800,0.212379,0.230337,0.357914,0.524689,0.180615,0.650000,0.139445,0.277113,0.283738,0.174050,0.608333
5,Q6,0.167832,0.461995,0.656287,0.150505,1,2,2,2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,None,1.000000,0.818966,1.000000,0.137838,0.818182,0.201553,0.431776,0.212980,0.197778,0.220602,0.130941,0.489542,0.252479,0.276800,0.212379,0.230337,0.357914,0.524689,0.180615,0.226630,0.139445,0.277113,0.283738,0.174050,0.429060
6,Q7,0.381437,0.904146,0.557748,0.945501,1,2,2,2,0.318182,0.636364,0.000000,0.000000,0.000000,0.090909,None,0.363636,0.893417,0.816456,0.529730,0.694215,0.201553,0.431776,0.212980,0.197778,0.000000,0.130941,0.489542,0.252479,0.276800,0.212379,0.216288,0.523125,0.524689,0.216288,0.226630,0.139445,0.277113,0.200000,0.174050,0.429060
7,Q8,0.538830,0.765002,0.531929,0.245561,1,2,2,2,0.552632,0.368421,0.274854,0.000000,0.000000,0.052632,None,0.000000,0.172414,0.681213,0.909246,0.918660,0.201553,0.431776,0.212980,0.197778,0.220602,0.000000,0.537500,0.252479,0.276800,0.212379,0.230337,0.533333,0.618019,0.180615,-0.250000,0.172078,0.277113,0.283738,0.174050,0.750000
8,Q9,0.820513,0.339503,0.073168,0.000000,0,2,2,2,0.388889,0.000000,0.000000,0.000000,0.137272,0.166667,None,0.111111,0.545977,0.775668,0.808408,0.727273,0.201553,0.431776,0.300000,0.300000,0.220602,0.130941,0.489542,0.050000,0.276800,0.193056,0.566667,0.342857,0.270941,0.450000,0.550000,0.139445,0.277113,0.212572,0.174050,0.496528
9,Q10,0.083916,0.473318,0.220711,0.602020,1,2,2,2,0.500000,0.500000,0.373016,0.204645,0.352986,0.142857,None,0.285714,0.935961,0.567360,0.384170,0.545455,0.201553,0.431776,-0.520000,-0.520000,0.220602,0.102083,0.489542,0.625000,0.276800,0.212379,0.226000,0.357914,0.580000,0.180615,-0.520000,0.139445,0.277113,0.283738,0.174050,0.560000


In [185]:
len(final)

40

In [186]:
earnings = [
10.28,
9.194,
10.39,
9.497,
11.22,
7.846,
5.178,
4.902,
7.349,
6.091,
2.616,
2.429,
6.882,
5.137,
5.106,
4.988,
4.269,
4.707,
3.894,
3.064,
3.569,
2.627,
2.283,
1.738,
1.561,
0.896,
0.719,
0.512,
0.701,
0.806,
0.791,
0.642,
0.523,
0.425,
0.333,
0.219,
0.64,
-0.59,
-0.157,
0.205
]

earnings = earnings[::-1]

In [187]:
len(earnings)

40

In [188]:
final['earnings'] = earnings

In [189]:
final

,quarters,ratings,title_senti,pnc_senti,work_duration,current_employee,recommend,ceo_approval,business_outlook,recommend_minus,ceo_minus,business_minus,recommend_unknown,ceo_unknown,business_unknown,recommend_same,ceo_same,business_same,recommend_plus,ceo_plus,business_plus,*_*,authentic_self,balance_life,balance_work,balance_work-life,balance_work/life,benefit_great,care_take,career_growth,culture_open,fast_pace,food_free,great_perk,internal_tool,life_work,menlo_park,none_none,people_smart,performance_review,place_work,earnings
0,Q1,0.951049,0.800650,0.840840,0.768193,1,2,2,0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,None,0.000000,0.000000,1.000000,1.000000,0.000000,0.201553,0.431776,0.212980,0.197778,0.220602,0.130941,0.452827,0.427083,0.276800,0.212379,0.478571,0.357914,0.639286,0.180615,0.226630,0.139445,0.277113,0.239286,0.174050,0.429060,0.205
1,Q2,0.839161,0.654483,0.706417,0.931246,1,2,2,2,0.500000,0.000000,0.000000,0.000000,0.000000,0.428571,None,0.000000,0.000000,0.711573,1.000000,0.623377,0.201553,0.431776,0.212980,0.197778,0.220602,0.130941,0.489542,0.252479,0.276800,0.212379,0.230337,0.357914,0.587500,0.180615,0.266667,0.139445,0.277113,0.283738,0.174050,0.062500,-0.157
2,Q3,0.831864,0.406544,0.606447,0.229029,1,2,2,2,0.152174,0.304348,0.227053,0.000000,0.000000,0.000000,None,0.000000,0.284858,0.912218,0.925029,0.948617,0.277778,0.431776,0.086111,0.014931,0.220602,0.130941,0.543750,0.252479,0.366667,0.212379,0.200000,0.308452,0.554857,0.180615,-0.027778,0.139445,0.277113,0.341929,0.352083,0.634766,-0.590
3,Q4,1.000000,0.777833,0.158513,0.402000,1,2,2,2,0.291667,0.000000,0.000000,0.119376,0.000000,0.041667,None,0.166667,0.136494,0.747627,0.856306,1.000000,0.201553,0.431776,0.520000,0.520000,0.220602,-0.062500,0.361756,0.252479,0.276800,0.200000,0.050000,0.207292,0.361756,0.180615,0.520000,0.139445,0.277113,0.283534,0.174050,0.550000,0.640
4,Q5,0.000000,0.456848,0.122679,0.537518,1,2,2,2,1.000000,1.000000,0.746032,0.000000,0.000000,0.000000,None,0.000000,0.467980,0.423146,0.753668,0.779221,0.201553,0.431776,0.212980,0.197778,0.220602,0.130941,0.489542,0.252479,0.276800,0.212379,0.230337,0.357914,0.524689,0.180615,0.650000,0.139445,0.277113,0.283738,0.174050,0.608333,0.219
5,Q6,0.167832,0.461995,0.656287,0.150505,1,2,2,2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,None,1.000000,0.818966,1.000000,0.137838,0.818182,0.201553,0.431776,0.212980,0.197778,0.220602,0.130941,0.489542,0.252479,0.276800,0.212379,0.230337,0.357914,0.524689,0.180615,0.226630,0.139445,0.277113,0.283738,0.174050,0.429060,0.333
6,Q7,0.381437,0.904146,0.557748,0.945501,1,2,2,2,0.318182,0.636364,0.000000,0.000000,0.000000,0.090909,None,0.363636,0.893417,0.816456,0.529730,0.694215,0.201553,0.431776,0.212980,0.197778,0.000000,0.130941,0.489542,0.252479,0.276800,0.212379,0.216288,0.523125,0.524689,0.216288,0.226630,0.139445,0.277113,0.200000,0.174050,0.429060,0.425
7,Q8,0.538830,0.765002,0.531929,0.245561,1,2,2,2,0.552632,0.368421,0.274854,0.000000,0.000000,0.052632,None,0.000000,0.172414,0.681213,0.909246,0.918660,0.201553,0.431776,0.212980,0.197778,0.220602,0.000000,0.537500,0.252479,0.276800,0.212379,0.230337,0.533333,0.618019,0.180615,-0.250000,0.172078,0.277113,0.283738,0.174050,0.750000,0.523
8,Q9,0.820513,0.339503,0.073168,0.000000,0,2,2,2,0.388889,0.000000,0.000000,0.000000,0.137272,0.166667,None,0.111111,0.545977,0.775668,0.808408,0.727273,0.201553,0.431776,0.300000,0.300000,0.220602,0.130941,0.489542,0.050000,0.276800,0.193056,0.566667,0.342857,0.270941,0.450000,0.550000,0.139445,0.277113,0.212572,0.174050,0.496528,0.642
9,Q10,0.083916,0.473318,0.220711,0.602020,1,2,2,2,0.500000,0.500000,0.373016,0.204645,0.352986,0.142857,None,0.285714,0.935961,0.567360,0.384170,0.545455,0.201553,0.431776,-0.520000,-0.520000,0.220602,0.102083,0.489542,0.625000,0.276800,0.212379,0.226000,0.357914,0.580000,0.180615,-0.520000,0.139445,0.277113,0.283738,0.174050,0.560000,0.791


In [190]:
pred_column = [None]
for i in range(1, len(earnings)):
	if earnings[i] > earnings[i-1]:
		pred_column.append(1)
	else:
		pred_column.append(0)

In [191]:
pred_column

[None,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1]

In [192]:
final['pred_column'] = pred_column

In [193]:
final

,quarters,ratings,title_senti,pnc_senti,work_duration,current_employee,recommend,ceo_approval,business_outlook,recommend_minus,ceo_minus,business_minus,recommend_unknown,ceo_unknown,business_unknown,recommend_same,ceo_same,business_same,recommend_plus,ceo_plus,business_plus,*_*,authentic_self,balance_life,balance_work,balance_work-life,balance_work/life,benefit_great,care_take,career_growth,culture_open,fast_pace,food_free,great_perk,internal_tool,life_work,menlo_park,none_none,people_smart,performance_review,place_work,earnings,pred_column
0,Q1,0.951049,0.800650,0.840840,0.768193,1,2,2,0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,None,0.000000,0.000000,1.000000,1.000000,0.000000,0.201553,0.431776,0.212980,0.197778,0.220602,0.130941,0.452827,0.427083,0.276800,0.212379,0.478571,0.357914,0.639286,0.180615,0.226630,0.139445,0.277113,0.239286,0.174050,0.429060,0.205,NaN
1,Q2,0.839161,0.654483,0.706417,0.931246,1,2,2,2,0.500000,0.000000,0.000000,0.000000,0.000000,0.428571,None,0.000000,0.000000,0.711573,1.000000,0.623377,0.201553,0.431776,0.212980,0.197778,0.220602,0.130941,0.489542,0.252479,0.276800,0.212379,0.230337,0.357914,0.587500,0.180615,0.266667,0.139445,0.277113,0.283738,0.174050,0.062500,-0.157,0.0
2,Q3,0.831864,0.406544,0.606447,0.229029,1,2,2,2,0.152174,0.304348,0.227053,0.000000,0.000000,0.000000,None,0.000000,0.284858,0.912218,0.925029,0.948617,0.277778,0.431776,0.086111,0.014931,0.220602,0.130941,0.543750,0.252479,0.366667,0.212379,0.200000,0.308452,0.554857,0.180615,-0.027778,0.139445,0.277113,0.341929,0.352083,0.634766,-0.590,0.0
3,Q4,1.000000,0.777833,0.158513,0.402000,1,2,2,2,0.291667,0.000000,0.000000,0.119376,0.000000,0.041667,None,0.166667,0.136494,0.747627,0.856306,1.000000,0.201553,0.431776,0.520000,0.520000,0.220602,-0.062500,0.361756,0.252479,0.276800,0.200000,0.050000,0.207292,0.361756,0.180615,0.520000,0.139445,0.277113,0.283534,0.174050,0.550000,0.640,1.0
4,Q5,0.000000,0.456848,0.122679,0.537518,1,2,2,2,1.000000,1.000000,0.746032,0.000000,0.000000,0.000000,None,0.000000,0.467980,0.423146,0.753668,0.779221,0.201553,0.431776,0.212980,0.197778,0.220602,0.130941,0.489542,0.252479,0.276800,0.212379,0.230337,0.357914,0.524689,0.180615,0.650000,0.139445,0.277113,0.283738,0.174050,0.608333,0.219,0.0
5,Q6,0.167832,0.461995,0.656287,0.150505,1,2,2,2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,None,1.000000,0.818966,1.000000,0.137838,0.818182,0.201553,0.431776,0.212980,0.197778,0.220602,0.130941,0.489542,0.252479,0.276800,0.212379,0.230337,0.357914,0.524689,0.180615,0.226630,0.139445,0.277113,0.283738,0.174050,0.429060,0.333,1.0
6,Q7,0.381437,0.904146,0.557748,0.945501,1,2,2,2,0.318182,0.636364,0.000000,0.000000,0.000000,0.090909,None,0.363636,0.893417,0.816456,0.529730,0.694215,0.201553,0.431776,0.212980,0.197778,0.000000,0.130941,0.489542,0.252479,0.276800,0.212379,0.216288,0.523125,0.524689,0.216288,0.226630,0.139445,0.277113,0.200000,0.174050,0.429060,0.425,1.0
7,Q8,0.538830,0.765002,0.531929,0.245561,1,2,2,2,0.552632,0.368421,0.274854,0.000000,0.000000,0.052632,None,0.000000,0.172414,0.681213,0.909246,0.918660,0.201553,0.431776,0.212980,0.197778,0.220602,0.000000,0.537500,0.252479,0.276800,0.212379,0.230337,0.533333,0.618019,0.180615,-0.250000,0.172078,0.277113,0.283738,0.174050,0.750000,0.523,1.0
8,Q9,0.820513,0.339503,0.073168,0.000000,0,2,2,2,0.388889,0.000000,0.000000,0.000000,0.137272,0.166667,None,0.111111,0.545977,0.775668,0.808408,0.727273,0.201553,0.431776,0.300000,0.300000,0.220602,0.130941,0.489542,0.050000,0.276800,0.193056,0.566667,0.342857,0.270941,0.450000,0.550000,0.139445,0.277113,0.212572,0.174050,0.496528,0.642,1.0
9,Q10,0.083916,0.473318,0.220711,0.602020,1,2,2,2,0.500000,0.500000,0.373016,0.204645,0.352986,0.142857,None,0.285714,0.935961,0.567360,0.384170,0.545455,0.201553,0.431776,-0.520000,-0.520000,0.220602,0.102083,0.489542,0.625000,0.276800,0.212379,0.226000,0.357914,0.580000,0.180615,-0.520000,0.139445,0.277113,0.283738,0.174050,0.560000,0.791,1.0


In [194]:
final.drop(['recommend_same'], axis=1, inplace=True)

In [195]:
final

,quarters,ratings,title_senti,pnc_senti,work_duration,current_employee,recommend,ceo_approval,business_outlook,recommend_minus,ceo_minus,business_minus,recommend_unknown,ceo_unknown,business_unknown,ceo_same,business_same,recommend_plus,ceo_plus,business_plus,*_*,authentic_self,balance_life,balance_work,balance_work-life,balance_work/life,benefit_great,care_take,career_growth,culture_open,fast_pace,food_free,great_perk,internal_tool,life_work,menlo_park,none_none,people_smart,performance_review,place_work,earnings,pred_column
0,Q1,0.951049,0.800650,0.840840,0.768193,1,2,2,0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.201553,0.431776,0.212980,0.197778,0.220602,0.130941,0.452827,0.427083,0.276800,0.212379,0.478571,0.357914,0.639286,0.180615,0.226630,0.139445,0.277113,0.239286,0.174050,0.429060,0.205,NaN
1,Q2,0.839161,0.654483,0.706417,0.931246,1,2,2,2,0.500000,0.000000,0.000000,0.000000,0.000000,0.428571,0.000000,0.000000,0.711573,1.000000,0.623377,0.201553,0.431776,0.212980,0.197778,0.220602,0.130941,0.489542,0.252479,0.276800,0.212379,0.230337,0.357914,0.587500,0.180615,0.266667,0.139445,0.277113,0.283738,0.174050,0.062500,-0.157,0.0
2,Q3,0.831864,0.406544,0.606447,0.229029,1,2,2,2,0.152174,0.304348,0.227053,0.000000,0.000000,0.000000,0.000000,0.284858,0.912218,0.925029,0.948617,0.277778,0.431776,0.086111,0.014931,0.220602,0.130941,0.543750,0.252479,0.366667,0.212379,0.200000,0.308452,0.554857,0.180615,-0.027778,0.139445,0.277113,0.341929,0.352083,0.634766,-0.590,0.0
3,Q4,1.000000,0.777833,0.158513,0.402000,1,2,2,2,0.291667,0.000000,0.000000,0.119376,0.000000,0.041667,0.166667,0.136494,0.747627,0.856306,1.000000,0.201553,0.431776,0.520000,0.520000,0.220602,-0.062500,0.361756,0.252479,0.276800,0.200000,0.050000,0.207292,0.361756,0.180615,0.520000,0.139445,0.277113,0.283534,0.174050,0.550000,0.640,1.0
4,Q5,0.000000,0.456848,0.122679,0.537518,1,2,2,2,1.000000,1.000000,0.746032,0.000000,0.000000,0.000000,0.000000,0.467980,0.423146,0.753668,0.779221,0.201553,0.431776,0.212980,0.197778,0.220602,0.130941,0.489542,0.252479,0.276800,0.212379,0.230337,0.357914,0.524689,0.180615,0.650000,0.139445,0.277113,0.283738,0.174050,0.608333,0.219,0.0
5,Q6,0.167832,0.461995,0.656287,0.150505,1,2,2,2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.818966,1.000000,0.137838,0.818182,0.201553,0.431776,0.212980,0.197778,0.220602,0.130941,0.489542,0.252479,0.276800,0.212379,0.230337,0.357914,0.524689,0.180615,0.226630,0.139445,0.277113,0.283738,0.174050,0.429060,0.333,1.0
6,Q7,0.381437,0.904146,0.557748,0.945501,1,2,2,2,0.318182,0.636364,0.000000,0.000000,0.000000,0.090909,0.363636,0.893417,0.816456,0.529730,0.694215,0.201553,0.431776,0.212980,0.197778,0.000000,0.130941,0.489542,0.252479,0.276800,0.212379,0.216288,0.523125,0.524689,0.216288,0.226630,0.139445,0.277113,0.200000,0.174050,0.429060,0.425,1.0
7,Q8,0.538830,0.765002,0.531929,0.245561,1,2,2,2,0.552632,0.368421,0.274854,0.000000,0.000000,0.052632,0.000000,0.172414,0.681213,0.909246,0.918660,0.201553,0.431776,0.212980,0.197778,0.220602,0.000000,0.537500,0.252479,0.276800,0.212379,0.230337,0.533333,0.618019,0.180615,-0.250000,0.172078,0.277113,0.283738,0.174050,0.750000,0.523,1.0
8,Q9,0.820513,0.339503,0.073168,0.000000,0,2,2,2,0.388889,0.000000,0.000000,0.000000,0.137272,0.166667,0.111111,0.545977,0.775668,0.808408,0.727273,0.201553,0.431776,0.300000,0.300000,0.220602,0.130941,0.489542,0.050000,0.276800,0.193056,0.566667,0.342857,0.270941,0.450000,0.550000,0.139445,0.277113,0.212572,0.174050,0.496528,0.642,1.0
9,Q10,0.083916,0.473318,0.220711,0.602020,1,2,2,2,0.500000,0.500000,0.373016,0.204645,0.352986,0.142857,0.285714,0.935961,0.567360,0.384170,0.545455,0.201553,0.431776,-0.520000,-0.520000,0.220602,0.102083,0.489542,0.625000,0.276800,0.212379,0.226000,0.357914,0.580000,0.180615,-0.520000,0.139445,0.277113,0.283738,0.174050,0.560000,0.791,1.0


In [196]:
final.dropna(inplace=True)

In [197]:
final.drop(['recommend_unknown', 'ceo_unknown', 'business_unknown'], axis=1, inplace=True)

In [198]:
final

,quarters,ratings,title_senti,pnc_senti,work_duration,current_employee,recommend,ceo_approval,business_outlook,recommend_minus,ceo_minus,business_minus,ceo_same,business_same,recommend_plus,ceo_plus,business_plus,*_*,authentic_self,balance_life,balance_work,balance_work-life,balance_work/life,benefit_great,care_take,career_growth,culture_open,fast_pace,food_free,great_perk,internal_tool,life_work,menlo_park,none_none,people_smart,performance_review,place_work,earnings,pred_column
1,Q2,0.839161,0.654483,0.706417,0.931246,1,2,2,2,0.500000,0.000000,0.000000,0.000000,0.000000,0.711573,1.000000,0.623377,0.201553,0.431776,0.212980,0.197778,0.220602,0.130941,0.489542,0.252479,0.276800,0.212379,0.230337,0.357914,0.587500,0.180615,0.266667,0.139445,0.277113,0.283738,0.174050,0.062500,-0.157,0.0
2,Q3,0.831864,0.406544,0.606447,0.229029,1,2,2,2,0.152174,0.304348,0.227053,0.000000,0.284858,0.912218,0.925029,0.948617,0.277778,0.431776,0.086111,0.014931,0.220602,0.130941,0.543750,0.252479,0.366667,0.212379,0.200000,0.308452,0.554857,0.180615,-0.027778,0.139445,0.277113,0.341929,0.352083,0.634766,-0.590,0.0
3,Q4,1.000000,0.777833,0.158513,0.402000,1,2,2,2,0.291667,0.000000,0.000000,0.166667,0.136494,0.747627,0.856306,1.000000,0.201553,0.431776,0.520000,0.520000,0.220602,-0.062500,0.361756,0.252479,0.276800,0.200000,0.050000,0.207292,0.361756,0.180615,0.520000,0.139445,0.277113,0.283534,0.174050,0.550000,0.640,1.0
4,Q5,0.000000,0.456848,0.122679,0.537518,1,2,2,2,1.000000,1.000000,0.746032,0.000000,0.467980,0.423146,0.753668,0.779221,0.201553,0.431776,0.212980,0.197778,0.220602,0.130941,0.489542,0.252479,0.276800,0.212379,0.230337,0.357914,0.524689,0.180615,0.650000,0.139445,0.277113,0.283738,0.174050,0.608333,0.219,0.0
5,Q6,0.167832,0.461995,0.656287,0.150505,1,2,2,2,0.000000,0.000000,0.000000,1.000000,0.818966,1.000000,0.137838,0.818182,0.201553,0.431776,0.212980,0.197778,0.220602,0.130941,0.489542,0.252479,0.276800,0.212379,0.230337,0.357914,0.524689,0.180615,0.226630,0.139445,0.277113,0.283738,0.174050,0.429060,0.333,1.0
6,Q7,0.381437,0.904146,0.557748,0.945501,1,2,2,2,0.318182,0.636364,0.000000,0.363636,0.893417,0.816456,0.529730,0.694215,0.201553,0.431776,0.212980,0.197778,0.000000,0.130941,0.489542,0.252479,0.276800,0.212379,0.216288,0.523125,0.524689,0.216288,0.226630,0.139445,0.277113,0.200000,0.174050,0.429060,0.425,1.0
7,Q8,0.538830,0.765002,0.531929,0.245561,1,2,2,2,0.552632,0.368421,0.274854,0.000000,0.172414,0.681213,0.909246,0.918660,0.201553,0.431776,0.212980,0.197778,0.220602,0.000000,0.537500,0.252479,0.276800,0.212379,0.230337,0.533333,0.618019,0.180615,-0.250000,0.172078,0.277113,0.283738,0.174050,0.750000,0.523,1.0
8,Q9,0.820513,0.339503,0.073168,0.000000,0,2,2,2,0.388889,0.000000,0.000000,0.111111,0.545977,0.775668,0.808408,0.727273,0.201553,0.431776,0.300000,0.300000,0.220602,0.130941,0.489542,0.050000,0.276800,0.193056,0.566667,0.342857,0.270941,0.450000,0.550000,0.139445,0.277113,0.212572,0.174050,0.496528,0.642,1.0
9,Q10,0.083916,0.473318,0.220711,0.602020,1,2,2,2,0.500000,0.500000,0.373016,0.285714,0.935961,0.567360,0.384170,0.545455,0.201553,0.431776,-0.520000,-0.520000,0.220602,0.102083,0.489542,0.625000,0.276800,0.212379,0.226000,0.357914,0.580000,0.180615,-0.520000,0.139445,0.277113,0.283738,0.174050,0.560000,0.791,1.0
10,Q11,0.786161,0.601173,0.587746,0.625784,1,2,2,2,0.184211,0.000000,0.000000,0.105263,0.689655,0.681213,0.727738,0.746411,0.201553,0.431776,-0.125000,-0.125000,0.800000,0.130941,0.438294,0.216667,0.276800,0.348333,0.230337,0.400000,0.800000,0.180615,-0.125000,0.100000,0.277113,0.283738,0.174050,0.400000,0.806,1.0


In [202]:
final.to_excel('meta_cleaned_data.xlsx')

In [203]:
final.to_csv('meta_cleaned_data.csv')